# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [12]:
nutrition_agent = Agent(
    name = "Nutrition Assistant",
    instructions="""
You are a helpful assistant giving out nutirtion advice. 
You give consice answers every single time. Activate economic budget version for tokens saving."""
)

Let's execute the Agent:

In [11]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are healthy: rich in potassium, vitamin C, vitamin B6, and fiber; low in fat. They’re convenient and help with energy. Note: higher sugar/GI when very ripe; portion for diabetes or low-sugar diets. Pair with protein/fat for balance.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [13]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are healthy in moderation. Key points:

- Pros: good potassium (supports blood pressure), vitamin B6, vitamin C, fiber; low fat; convenient.
- Cons: contains natural sugars; calories ~100 per medium banana.
- Tips: pair with protein/fat for fullness; choose a ripeness based on goals (ripe = sweeter, more sugar; greener = more resistant starch).
- Watch for: rare allergy; consider portions if monitoring sugar intake (diabetes).

Want a quick serving suggestion?

_Good Job!_